<a href="https://colab.research.google.com/github/bineeshpc/openvino_project/blob/master/transfer_learning_submit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import TensorFlow

In [ ]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import random

import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [ ]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()


In [ ]:
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(32, 32, 3),
    include_top=False)  # Do not include the ImageNet classifier at the top.

base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
inputs = tf.keras.Input(shape=(32, 32, 3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
# outputs = tf.keras.layers.Dense(10)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=25)
]

history = model.fit(train_images, train_labels, epochs=100, batch_size=32,
                    
                    
                     validation_data=(test_images, test_labels),
                   callbacks=my_callbacks)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
model_filename = "transfer_learning.h5"
model.save(model_filename)

In [ ]:
print(test_acc)

In [ ]:
from google.colab import files
files.download(model_filename) 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
# Unfreeze the base model
base_model.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2)
]

history = model.fit(train_images, train_labels, epochs=10, batch_size=32,
                    
                    
                     validation_data=(test_images, test_labels),
                   callbacks=my_callbacks)

Epoch 1/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.8115 - accuracy: 0.6475 - val_loss: 1.8265 - val_accuracy: 0.6332
Epoch 2/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7960 - accuracy: 0.6636 - val_loss: 1.7813 - val_accuracy: 0.6782
Epoch 3/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7802 - accuracy: 0.6797 - val_loss: 1.7700 - val_accuracy: 0.6900
Epoch 4/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7579 - accuracy: 0.7018 - val_loss: 1.7597 - val_accuracy: 0.6996
Epoch 5/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7456 - accuracy: 0.7142 - val_loss: 1.7426 - val_accuracy: 0.7170
Epoch 6/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7352 - accuracy: 0.7250 - val_loss: 1.7445 - val_accuracy: 0.7157
Epoch 7/10
1563/1563 [==============================] - 62s 40ms/step - loss: 1.7254 - accuracy: 0.7348 - val_loss: 1.7234 -

In [18]:
# Unfreeze the base model
base_model.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2)
]

history = model.fit(train_images, train_labels, epochs=10, batch_size=32,
                    
                    
                     validation_data=(test_images, test_labels),
                   callbacks=my_callbacks)

Epoch 1/10
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6922 - accuracy: 0.7684 - val_loss: 1.7053 - val_accuracy: 0.7550
Epoch 2/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6901 - accuracy: 0.7702 - val_loss: 1.7060 - val_accuracy: 0.7540
Epoch 3/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6929 - accuracy: 0.7673 - val_loss: 1.7049 - val_accuracy: 0.7558
Epoch 4/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6927 - accuracy: 0.7673 - val_loss: 1.7056 - val_accuracy: 0.7546
Epoch 5/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6890 - accuracy: 0.7718 - val_loss: 1.7039 - val_accuracy: 0.7566
Epoch 6/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6903 - accuracy: 0.7703 - val_loss: 1.7037 - val_accuracy: 0.7565
Epoch 7/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6881 - accuracy: 0.7723 - val_loss: 1.7038 -

In [19]:

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2)
]

history = model.fit(train_images, train_labels, epochs=10, batch_size=32,
                    
                    
                     validation_data=(test_images, test_labels),
                   callbacks=my_callbacks)

Epoch 1/10
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6883 - accuracy: 0.7723 - val_loss: 1.7008 - val_accuracy: 0.7596
Epoch 2/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6847 - accuracy: 0.7756 - val_loss: 1.6983 - val_accuracy: 0.7614
Epoch 3/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6841 - accuracy: 0.7765 - val_loss: 1.6959 - val_accuracy: 0.7644
Epoch 4/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6814 - accuracy: 0.7786 - val_loss: 1.6940 - val_accuracy: 0.7670
Epoch 5/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6817 - accuracy: 0.7789 - val_loss: 1.6934 - val_accuracy: 0.7672
Epoch 6/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6791 - accuracy: 0.7814 - val_loss: 1.6925 - val_accuracy: 0.7677
Epoch 7/10
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6787 - accuracy: 0.7818 - val_loss: 1.6915 -

In [20]:

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5)
]

history = model.fit(train_images, train_labels, epochs=50, batch_size=32,
                    
                    
                     validation_data=(test_images, test_labels),
                   callbacks=my_callbacks)

Epoch 1/50
1563/1563 [==============================] - 21s 13ms/step - loss: 1.6781 - accuracy: 0.7822 - val_loss: 1.6896 - val_accuracy: 0.7705
Epoch 2/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6762 - accuracy: 0.7842 - val_loss: 1.6872 - val_accuracy: 0.7728
Epoch 3/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6729 - accuracy: 0.7875 - val_loss: 1.6850 - val_accuracy: 0.7754
Epoch 4/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6706 - accuracy: 0.7897 - val_loss: 1.6848 - val_accuracy: 0.7749
Epoch 5/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6684 - accuracy: 0.7924 - val_loss: 1.6849 - val_accuracy: 0.7750
Epoch 6/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6671 - accuracy: 0.7937 - val_loss: 1.6826 - val_accuracy: 0.7779
Epoch 7/50
1563/1563 [==============================] - 20s 13ms/step - loss: 1.6658 - accuracy: 0.7942 - val_loss: 1.6840 -

In [21]:
model_filename = "transfer_learning1.h5"
model.save(model_filename)
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)
from google.colab import files
files.download(model_filename)

313/313 - 3s - loss: 1.6832 - accuracy: 0.7772
0.7771999835968018


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>